In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import logging 

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

import json
import polars as pl
from polars import col, lit
import altair as alt
import simrs
from history import History, tabular
import input_builder as b
import utils 

In [3]:
# Agents
agent_ids = ["a" +str(i) for i in range(1, 5)]

# Goods
wheat = "Wheat"
goods = [wheat]

# Ports
genoa = "Genoa"
rome = "Rome"
milan = "Milan"
venice = "Venice"
marsailles = "Marsailles"
port_ids = [genoa, rome, milan, venice, marsailles]

# genoa -> milan -> marsailles
#   v         v
# rome -> venice
edges = [(genoa, milan), (milan, marsailles), (rome, venice), (rome, genoa), (venice, milan)]
_market = lambda net: b._market(wheat, b._market_info(net=net)) 

# net balanced
ports = [
  b._port(genoa, _market(100)), 
  b._port(milan, _market(50)), 
  b._port(rome, _market(0)), 
  b._port(venice, _market(-50)),
  b._port(marsailles, _market(-100)), 
]

_agent = lambda id, pos: b._agent(id, pos, 1000, "Greedy")
agents = [_agent(id, port_id) for (id, port_id) in zip(agent_ids, port_ids)]

x = b._inputFormat( agents = agents, ports=ports, edges= edges, opts=b._opts(ticks=100))
# history = simrs.run(x)
# (actions, agents, markets) = tabular(history)

In [4]:
h = History.from_input(x)
h.run({"ticks": 100})

cost $1440.2 > wallet $1432.5999999999983


RuntimeError: Invalid Action: Tried to sell when impossible. Agent: Agent { id: AgentId(u!("a2")), pos: PortId(u!("Genoa")), cargo: Some(Good { name: u!("Wheat") }), coins: $1432.60, behavior: Greedy }, Action: Sell { good: Good { name: u!("Wheat") } })

Location:
    /rustc/8c74a5d27c644a0f7a22bb2fa8dd3ff8257bc220/library/core/src/convert/mod.rs:716:9

In [ ]:
# h.err
dir(h)
h.error()["InvalidAction"]["state"]["agents"]

{'a2': {'id': 'a2',
  'pos': 'Milan',
  'cargo': None,
  'coins': 777.6000000000017,
  'behavior': 'Greedy'},
 'a1': {'id': 'a1',
  'pos': 'Genoa',
  'cargo': None,
  'coins': 7.600000000001728,
  'behavior': 'Greedy'},
 'a4': {'id': 'a4',
  'pos': 'Genoa',
  'cargo': 'Wheat',
  'coins': 1462.3999999999983,
  'behavior': 'Greedy'},
 'a3': {'id': 'a3',
  'pos': 'Milan',
  'cargo': 'Wheat',
  'coins': 1482.3999999999983,
  'behavior': 'Greedy'}}

In [ ]:
e = h.error()["InvalidAction"]
print(e["action"])
print(e["agent"])
print(e["state"]["agents"])
print(e["state"]["ports"]["Genoa"])
utils.price(e["state"]["ports"]["Genoa"]["market"]["table"]["Wheat"])

{'action': 'Sell', 'good': 'Wheat'}
{'id': 'a2', 'pos': 'Genoa', 'cargo': 'Wheat', 'coins': 1432.5999999999983, 'behavior': 'Greedy'}
{'a3': {'id': 'a3', 'pos': 'Genoa', 'cargo': None, 'coins': 37.40000000000168, 'behavior': 'Greedy'}, 'a2': {'id': 'a2', 'pos': 'Genoa', 'cargo': 'Wheat', 'coins': 1432.5999999999983, 'behavior': 'Greedy'}, 'a1': {'id': 'a1', 'pos': 'Milan', 'cargo': 'Wheat', 'coins': 1477.4000000000017, 'behavior': 'Greedy'}, 'a4': {'id': 'a4', 'pos': 'Milan', 'cargo': 'Wheat', 'coins': 1452.5999999999983, 'behavior': 'Greedy'}}
{'id': 'Genoa', 'market': {'table': {'Wheat': {'consumption': 100.0, 'supply': 8700.0, 'production': 200.0, 'pricer': {'base_supply': 1000.0, 'base_price': 100.0, 'price_per_supply': -0.2}}}}}


-1440.0

In [ ]:
h.error()["InvalidAction"]["state"]["ports"]["Genoa"]

{'id': 'Genoa',
 'market': {'table': {'Wheat': {'consumption': 100.0,
    'supply': 8900.0,
    'production': 200.0,
    'pricer': {'base_supply': 1000.0,
     'base_price': 100.0,
     'price_per_supply': -0.2}}}}}

In [ ]:
h.history["states"][-1]["agents"]

{'a2': {'id': 'a2',
  'pos': 'Milan',
  'cargo': 'Wheat',
  'coins': 1467.6000000000017,
  'behavior': 'Greedy'},
 'a1': {'id': 'a1',
  'pos': 'Genoa',
  'cargo': 'Wheat',
  'coins': 1487.6000000000017,
  'behavior': 'Greedy'},
 'a4': {'id': 'a4',
  'pos': 'Genoa',
  'cargo': 'Wheat',
  'coins': 1462.3999999999983,
  'behavior': 'Greedy'},
 'a3': {'id': 'a3',
  'pos': 'Milan',
  'cargo': 'Wheat',
  'coins': 1482.3999999999983,
  'behavior': 'Greedy'}}

In [ ]:
simrs.bar()

bad 2
sus 2


'hi'

In [ ]:
markets.select(col("pricer").struct.field("base_price")).head()
markets.select(col("pricer")).schema

In [ ]:
def plot_agents(agents: pl.DataFrame):
    base = alt.Chart(agents).encode(
        x='tick:Q',
        y=alt.Y('coins:Q').scale(zero=False),
        color='id:O',
    )
    lines = base.transform_loess('tick', 'coins', bandwidth=.5, groupby=['id']).mark_line(size=4)
    return (base.mark_point() + lines).interactive() 
  
plot_agents(agents)

alt.LayerChart(...)

In [ ]:
class DotDict(dict):
    def __getattr__(self, name):
        return self.get(name, None)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]

def recursive_dotdict(obj):
    if isinstance(obj, dict):
        return DotDict({k: recursive_dotdict(v) for k, v in obj.items()})
    elif isinstance(obj, list):
        return [recursive_dotdict(element) for element in obj]
    else:
        return obj

# Test the function
nested_dict = {'a': 1, 'b': {'c': 2, 'd': {'e': 3}}, 'f': [1, {'g': 4}]}
nested_dot_dict = recursive_dotdict(nested_dict)

print(nested_dot_dict.b.d.e)  # Should print 3
print(nested_dot_dict.f[1].g)  # Should print 4

3
4


In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


Metrics for individual agent
- total coins
- coins per tick
- died?

Metrics for population of agents
- min, max, median, mean, std of agent coins

How well did the agents equalize prices?
- box plot of prices
- stddev of pricesf

Construct 'trades'
- bought Cargo at StartPort for BuyPrice
- sold Cargo at EndPort for SellPrice
- profit = SellPrice - BuyPrice
- Route = StartPort -> .. ->  EndPort
- RouteLength = len(Route)